In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [29]:
from collections import Counter
from datetime import datetime
from pathlib import Path
import pickle
import sys
from tqdm import tqdm

from rdkit import Chem

import matplotlib.pyplot as plt
from pymatgen.core.structure import Molecule

Custom plotting code... ignore this if you don't know what it is, it should gracefully do nothing if you don't have the `MPLAdjutant` class. 

In [3]:
sys.path.append(str(Path.home() / Path("local")))
class NullClass:
    def do_nothing(*args, **kwargs):
        pass
    def add_colorbar(self, im, **kwargs):
        return plt.colorbar(im)
    def __getattr__(self, _):
        return self.do_nothing
try:
    from mpl_utils import MPLAdjutant
    adj = MPLAdjutant()
    adj.set_defaults()
except ImportError:
    adj = NullClass()

Append the `home` path of this project.

In [4]:
sys.path.append(str(Path.cwd().parent))

# Load the pickled data

Load in the previously processed `pickle` data. These contain the features (ACSF) and targets (spectra) on a common grid for each of the three absorbing atom types of interest: C, N and O.

In [5]:
O_data = pickle.load(open("../data/qm9/XANES-220622-ACSF-O.pkl", "rb"))
N_data = pickle.load(open("../data/qm9/XANES-220622-ACSF-N.pkl", "rb"))
C_data = pickle.load(open("../data/qm9/XANES-220622-ACSF-C.pkl", "rb"))

Print some diagonstics...

In [6]:
print(f"O database ACSF & spectrum shapes: {O_data['x'].shape} & {O_data['y'].shape}")
print(f"N database ACSF & spectrum shapes: {N_data['x'].shape} & {N_data['y'].shape}")
print(f"C database ACSF & spectrum shapes: {C_data['x'].shape} & {C_data['y'].shape}")

O database ACSF & spectrum shapes: (186813, 155) & (186813, 200)
N database ACSF & spectrum shapes: (139058, 155) & (139058, 200)
C database ACSF & spectrum shapes: (841912, 155) & (841912, 200)


Put everything in a dictionary for simplicity.

In [7]:
data = {"O": O_data, "N": N_data, "C": C_data}

# Start parsing

## Option 1: train on molecules with at most two absorbers

In [62]:
from xas_nne.qm9 import split_qm9_data_by_number_of_absorbers

In [63]:
parsed_data = split_qm9_data_by_number_of_absorbers(data["N"], absorber="N")

Parsing the qm9 data by number of absorbers=N
Training data will have <=2 absorbers
Test data will get the rest
Keeping zwitterions: False
Computing smiles_to_n_absorbers_map


100%|███████████████████████████████████| 82559/82559 [00:56<00:00, 1449.01it/s]
